In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.rename(columns = {'Population (2016 est.)[8]':'population'}, inplace = True)

,Metropolitan area,population,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


In [24]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#clean up cities nhl team names and pop
cities.rename(columns = {'Population (2016 est.)[8]':'population'}, inplace = True)
cities['NHL'].replace('\[.*\]','', regex=True, inplace = True)
cities['NHL'].replace('\—', '', regex=True, inplace = True)

#clean up the nhl data
nhl_df = nhl_df[nhl_df['year']==2018]
nhl_df['W'] = nhl_df['W'].astype(str)
nhl_df = nhl_df[-nhl_df['W'].str.contains('Division')]
nhl_df['team'].replace('\*', '', regex=True,inplace = True)

#finds the name of the team from the entire term
nhl_df['name'] = nhl_df['team'].apply(lambda x: re.findall('(\w+)$', x)).astype('str')

#finds the index from the cities df to merge on based on team name
nhl_df['index'] = nhl_df['name'].apply(lambda x: cities['NHL'].str.find(x.strip('\[\]\'')).idxmax())
nhl_df = nhl_df.set_index('index').sort_values('index')

#Merge the popluation data to the nhl data
nhl_df = nhl_df.merge(cities[['Metropolitan area','population']], left_index = True, right_index = True)

#changes the data type for the wins and losses
nhl_df = nhl_df.explode('population')
nhl_df['population'] = nhl_df['population'].astype(float)
nhl_df['W'] = nhl_df['W'].astype(float)
nhl_df['L'] = nhl_df['L'].astype(float)
print( nhl_df)


#aggreagate data on metropolitan ar
win_loss_by_region = nhl_df.groupby('Metropolitan area').apply(lambda x: (x['W']/(x['W']+x['L'])).mean())
population_by_region = nhl_df.groupby('Metropolitan area').apply(lambda x: (x['population']).mean())
population_by_region = population_by_region.astype(int)
stats.pearsonr(population_by_region, win_loss_by_region)[0]
pd.DataFrame({'pop':population_by_region, 'win': win_loss_by_region})

                     team  GP     W     L  OL  PTS  PTS%   GF   GA    SRS  \
0        New York Rangers  82  34.0  39.0   9   77  .470  231  268  -0.46   
0        New York Rangers  82  34.0  39.0   9   77  .470  231  268  -0.46   
0        New York Rangers  82  34.0  39.0   9   77  .470  231  268  -0.46   
0       New Jersey Devils  82  44.0  29.0   9   97  .591  248  244   0.02   
0       New Jersey Devils  82  44.0  29.0   9   97  .591  248  244   0.02   
0       New Jersey Devils  82  44.0  29.0   9   97  .591  248  244   0.02   
0      New York Islanders  82  35.0  37.0  10   80  .488  264  296  -0.40   
0      New York Islanders  82  35.0  37.0  10   80  .488  264  296  -0.40   
0      New York Islanders  82  35.0  37.0  10   80  .488  264  296  -0.40   
1       Los Angeles Kings  82  45.0  29.0   8   98  .598  239  203   0.44   
1       Los Angeles Kings  82  45.0  29.0   8   98  .598  239  203   0.44   
1           Anaheim Ducks  82  44.0  25.0  13  101  .616  235  216   0.24   

,pop,win
Metropolitan area,,
Boston,4794447,0.714286
Buffalo,1132804,0.357143
Calgary,1392609,0.513889
Chicago,9512999,0.458333
Columbus,2041520,0.600000
Dallas–Fort Worth,7233323,0.567568
Denver,2853077,0.589041
Detroit,4297617,0.434783
Edmonton,1321426,0.473684


In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#clean up cities data
cities['NBA'].replace('\[.*\]','', regex=True, inplace = True)
cities['NBA'].replace('\—', '', regex=True, inplace = True)

#clean up nba data
nba_df = nba_df[nba_df['year']==2018]
nba_df['team'].replace('\**\s\(\d+\)', '', regex=True,inplace = True)

#splits the team name off and creates index to merge dfs
nba_df['name'] = nba_df['team'].apply(lambda x: re.findall('(\w+)$', x)).astype('str')
nba_df['index'] = nba_df['name'].apply(lambda x: cities['NBA'].str.find(x.strip('\[\]\'')).idxmax())
nba_df = nba_df.set_index('index').sort_values('index')

#Merge the dfs
nba_df = nba_df.merge(cities[['Metropolitan area','Population (2016 est.)[8]']], left_index = True, right_index = True)

#aggregate the data
nba_df['W/L%'] = nba_df['W/L%'].astype(float)
nba_df['Population (2016 est.)[8]'] = nba_df['Population (2016 est.)[8]'].astype(int)
win_loss_by_region = nba_df.groupby('Metropolitan area').apply(lambda x: (x['W/L%']).mean())
population_by_region = nba_df.groupby('Metropolitan area').apply(lambda x: (x['Population (2016 est.)[8]']).mean())
stats.pearsonr(population_by_region, win_loss_by_region)[0]

-0.17636350642182938

In [26]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#clean up cities nhl team names and pop
cities['MLB'].replace('\[.*\]','', regex=True, inplace = True)
cities['MLB'].replace('\—', '', regex=True, inplace = True)

#clean up the mlb data
mlb_df = mlb_df[mlb_df['year']==2018]

#finds the name of the team from the entire term
mlb_df['name'] = mlb_df['team'].apply(lambda x: re.findall('(\w+)$', x)).astype('str')
#requires an additional step to differentiate between the red and white sox
mlb_df['name'][mlb_df['name']=="['Sox']"] = mlb_df[mlb_df['name']=="['Sox']"]['team'].apply(lambda x: re.findall('(\w+\s\w+)$', x)).astype('str')

#finds the index from the cities df to merge on based on team name
mlb_df['index'] = mlb_df['name'].apply(lambda x: cities['MLB'].str.find(x.strip('\[\]\'')).idxmax())
mlb_df = mlb_df.set_index('index').sort_values('index')

#Merge the popluation data to the nhl data
mlb_df = mlb_df.merge(cities[['Metropolitan area','Population (2016 est.)[8]']], left_index = True, right_index = True)

#changes the data type for the wins and losses
mlb_df['W'] = mlb_df['W'].astype(int)
mlb_df['L'] = mlb_df['L'].astype(int)
mlb_df['Population (2016 est.)[8]'] = mlb_df['Population (2016 est.)[8]'].astype(int)

#aggreagate data on metropolitan ar
win_loss_by_region = mlb_df.groupby('Metropolitan area').apply(lambda x: (x['W']/(x['L']+x['W'])).mean())
population_by_region = mlb_df.groupby('Metropolitan area').apply(lambda x: (x['Population (2016 est.)[8]']).mean())
print(population_by_region)
print(win_loss_by_region)

Metropolitan area
Atlanta                    5789700.0
Baltimore                  2798886.0
Boston                     4794447.0
Chicago                    9512999.0
Cincinnati                 2165139.0
Cleveland                  2055612.0
Dallas–Fort Worth          7233323.0
Denver                     2853077.0
Detroit                    4297617.0
Houston                    6772470.0
Kansas City                2104509.0
Los Angeles               13310447.0
Miami–Fort Lauderdale      6066387.0
Milwaukee                  1572482.0
Minneapolis–Saint Paul     3551036.0
New York City             20153634.0
Philadelphia               6070500.0
Phoenix                    4661537.0
Pittsburgh                 2342299.0
San Diego                  3317749.0
San Francisco Bay Area     6657982.0
Seattle                    3798902.0
St. Louis                  2807002.0
Tampa Bay Area             3032171.0
Toronto                    5928040.0
Washington, D.C.           6131977.0
dtype: float64
Metro

In [21]:
stats.pearsonr(population_by_region, win_loss_by_region)

(0.15027698302669307, 0.46370703378875583)

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

#clean up cities nhl team names and pop
cities['NFL'].replace('\[.*\]','', regex=True, inplace = True)
cities['NFL'].replace('\—', '', regex=True, inplace = True)

#clean up the mlb data
nfl_df = nfl_df[nfl_df['year']==2018]
nfl_df['W'] = nfl_df['W'].astype(str)
nfl_df = nfl_df[-nfl_df['W'].str.contains('\D', regex=True)]

#finds the team name
nfl_df['team'] = nfl_df['team'].replace('[\*\+]', '', regex= True)
nfl_df['name'] = nfl_df['team'].apply(lambda x: re.findall('(\w+)$', x)).astype('str')

#creates the index to join
nfl_df['index'] = nfl_df['name'].apply(lambda x: cities['NFL'].str.find(x.strip('\[\]\'')).idxmax())
nfl_df = nfl_df.set_index('index').sort_values('index')

#joins the two dfs
nfl_df = nfl_df.merge(cities[['Metropolitan area','Population (2016 est.)[8]']], left_index = True, right_index = True)

#changes the data type of wins and loses
nfl_df['W'] = nfl_df['W'].astype(int)
nfl_df['L'] = nfl_df['L'].astype(int)
nfl_df['Population (2016 est.)[8]'] = nfl_df['Population (2016 est.)[8]'].astype(int)

#aggregates the results
win_loss_by_region = nfl_df.groupby('Metropolitan area').apply(lambda x: (x['W']/(x['W']+x['L'])).mean())
population_by_region = nfl_df.groupby('Metropolitan area').apply(lambda x: (x['Population (2016 est.)[8]']).mean())
stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.004922112149349393

In [5]:
nfl_df.columns

Index(['DSRS', 'L', 'League', 'MoV', 'OSRS', 'PA', 'PD', 'PF', 'SRS', 'SoS',
       'T', 'W', 'W-L%', 'team', 'year', 'name', 'Metropolitan area',
       'Population (2016 est.)[8]'],
      dtype='object')

In [42]:
import numpy as np

a = np.arange(8)
b = a[4:6]
b[:] = 40
c = a[4] + a[6]
c

46

In [35]:
import re
s = 'ABCAC'
bool(re.match('A', s)) == True

True

In [28]:
'[^A](?=AAA)'

array([0, 1, 2, 3, 4, 5, 6, 7])

In [41]:
df = pd.Series({'d':4, 'b':7, 'a':-5, 'c':3})
df.index[0]

4